# ABOR example

This notebook will first setup all the supporting data (i.e portfolios, instruments, transactions, quotes). We will then do the following:

1) Create a chart of accounts
2) Add charts to chart of accounts
3) Create posting rules
4) Create ABOR configuration
5) Create ABOR
6) Create journal entry lines
7) Create GL profile mappings
8) Create GL profile
9) Create trial balance
10) Run trial balance check


To run the demo, you can run the notebook to create all the supporting configuration. this will create a folder structure in Drive to upload files. Note only the market data are uploaded to te drive due to their large size. 

## Setup 
First we will create the valuation recipe and transaction types which are required for the commands.

In [2]:
# Import general purpose packages
import json
import os
import time
import pandas as pd
import logging
import argparse

# Import LUSID specific packages
from lusid.utilities import ApiClientFactory as LusidApiClientFactory
import lumipy
import lusid
import lusid.api as la
import lusid.models as models
import lusid_drive
from lusid_workflow.rest import ApiException
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidjam import RefreshingToken

# Create loggers
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# Authenticate our user and create API client
secrets_path = os.getenv("FBN_SECRETS_PATH")
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook"
)

# Get URL and use it to configure Drive SDKs
configuration_recipe_api = api_factory.build(la.ConfigurationRecipeApi)
api_url = api_factory.api_client.configuration._base_path.replace("api","")
drive_configuration = lusid_drive.Configuration(host=api_url + "drive")
drive_configuration.access_token = api_factory.api_client.configuration.access_token
drive_configuration = lusid_drive.Configuration(
    host=f"{configuration_recipe_api.api_client.configuration.host[:-4]}/drive"
)
drive_configuration.access_token = (
    configuration_recipe_api.api_client.configuration.access_token
)
with lusid_drive.ApiClient(drive_configuration) as api_client:
    files_api = lusid_drive.FilesApi(api_client)
    folders_api = lusid_drive.FoldersApi(api_client)
# Setup Drive, and Lumipy Clients
drive_client = lusid_drive.ApiClient(drive_configuration)

def run_luminesce(luminesce_string):  
    client = lumipy.get_client(api_secrets_filename=secrets_path, token=RefreshingToken())
    df = client.query_and_fetch(luminesce_string)
    return df

In [2]:
# Attempt to create the folder structure in LUSID Drive
# All files will live in a 'abor-demo-data' parent folder and then 'trades', 'prices' and 'source-data' subfolders.
drive_parent_folder='abor-demo-data'
drive_price_subfolder = "prices"
drive_coa_subfolder='COA'
local_filepath = "./data/"


def create_folder_drive(path, name):
    try:
        response = folders_api.create_folder(
            lusid_drive.models.CreateFolder(path=path, name=name)
        )
        print(f"Folder '{path}{name}' created successfully!")
    except lusid_drive.rest.ApiException as e:
        if json.loads(e.body)["code"] == 664:
            print(f"Folder '{path}{name}' already exists in Drive.")
        else:
            raise


create_folder_drive("/", drive_parent_folder)
create_folder_drive(f"/{drive_parent_folder}/", drive_price_subfolder)
create_folder_drive(f"/{drive_parent_folder}/", drive_coa_subfolder)




# Define function that will upload our files into Drive from our local data subfolder
def upload_files_drive(subfolder, filename):
    try:
        with open(local_filepath + filename, "rb") as data:
            response = files_api.create_file(
                x_lusid_drive_filename=filename,
                x_lusid_drive_path=drive_parent_folder + "/" + subfolder,
                content_length=os.stat(local_filepath + filename).st_size,
                body=data.read(),
            )
            print(f"File '/{drive_parent_folder}/{subfolder}/{filename}' uploaded successfully!")
    except lusid_drive.rest.ApiException as e:
        if json.loads(e.body)["code"] == 671:
            print(f"File '/{drive_parent_folder}/{subfolder}/{filename}' already exists in Drive.")
        else:
            raise


file_details = {
    "aborquotes.xlsx": drive_price_subfolder,
    "daily_quotes.xlsx": drive_price_subfolder,
    'chart-of-accounts.csv':drive_coa_subfolder,
}

for k, v in file_details.items():
    upload_files_drive(subfolder=v, filename=k)

Folder '/abor-demo-data' already exists in Drive.
Folder '/abor-demo-data/prices' already exists in Drive.
Folder '/abor-demo-data/COA' already exists in Drive.
File '/abor-demo-data/prices/aborquotes.xlsx' already exists in Drive.
File '/abor-demo-data/prices/daily_quotes.xlsx' already exists in Drive.
File '/abor-demo-data/COA/chart-of-accounts.csv' already exists in Drive.


### Prerequisite configuration

Before we set up all the ABOR data and configuration, we need to create some LUSID configuration to support the logic we want to build. We will set up a simple valuation recipe and all the necessary transaction types for this workflow. 

In [ ]:
def create_recipe(api_factory):
    recipes_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

    scope = "FBN_Funds"
    recipe_code = "marketValue"

    configuration_recipe = models.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="10D.0D",
                ),
                models.MarketDataKeyRule(
                    key="FX.*.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="10D.0D",
                ),
            ],
            suppliers=models.MarketContextSuppliers(
                commodity="Client",
                credit="Client",
                equity="Client",
                fx="Client",
                rates="Client",
            ),
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="ClientInternal",
                default_scope=scope,
                attempt_to_infer_missing_fx=True,
            ),
        ),
        pricing=models.PricingContext(
            model_rules=[
                models.VendorModelRule(
                    supplier="Lusid", model_name="SimpleStatic", instrument_type="Bond"
                )
            ]
        ),
    )

    upsert_configuration_recipe_response = recipes_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )

    print(upsert_configuration_recipe_response)


def create_txn_types(api_factory):

    scope = "FBN_Funds"
    
    system_configuration = api_factory.build(lusid.api.SystemConfigurationApi)

    response = system_configuration.set_transaction_configuration_source(
        source="abor",
        set_transaction_configuration_source_request=[
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FundsIn",
                        description="Deposit New Funds",
                        transaction_class="CashTransfers",
                        transaction_group="abor",
                        transaction_roles="Longer",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Subscription",
                        movement_types="CashReceivable",
                        side="Side1",
                        direction=1,
                        movement_options=["Capital"],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )
                        ]
                    )
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FundsOut",
                        description="Deposit New Funds",
                        transaction_class="CashTransfers",
                        transaction_group="abor",
                        transaction_roles="Shorter",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Redemption",
                        movement_types="CashReceivable",
                        side="Side1",
                        direction=-1,
                        movement_options=["Capital"],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )]),
                
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="CashIn",
                        description="New cash into portfolio",
                        transaction_class="CashTransfers",
                        transaction_group="abor",
                        transaction_roles="Longer",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="CashIn",
                        movement_types="CashReceivable",
                        movement_options=["Capital"],
                        side="Side1",
                        direction=1
                    )
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="Buy",
                        description="Buy",
                        transaction_class="Buy",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Bought",
                        movement_types="StockMovement",
                        side="Side1",
                        direction=1,
                        movement_options=[],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="NonCashInvestments"
                        
                        )]
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="CashInvested",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=-1,
                        movement_options=[],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )]
                        ,
                        
                        
                    ),
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="Sell",
                        description="Sell",
                        transaction_class="Sell",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Sale",
                        movement_types="StockMovement",
                        side="Side1",
                        direction=-1,
                        movement_options=[],
                         mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="NonCashInvestments"
                        
                        )]
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="CashProceeds",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=1,
                        movement_options=[],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )]
                    ),
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FxSpotBuy",
                        description="FxSpotBuy",
                        transaction_class="FxSpotBuy",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="FxSpotBuyLeg",
                        movement_types="CashCommitment",
                        side="Side1",
                        direction=1,
                        movement_options=[],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )]
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="FxSpotSellLeg",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=-1,
                        movement_options=[],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )]
                    ),
                ],
            ),
        ],
    )
    
    return response


if __name__ == "__main__":
    token = RefreshingToken()

    if token is not None:
        lusid_api_factory = LusidApiClientFactory(token=token)

    else:
        ap = argparse.ArgumentParser()
        ap.add_argument("-s", "--secrets", type=str, help="full path to json file")
        args = ap.parse_args()
        secrets_file = args.secrets
        lusid_api_factory = LusidApiClientFactory(api_secrets_filename=secrets_file)

    create_recipe(lusid_api_factory)
    create_txn_types(lusid_api_factory)

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-oscar.lusid.com/app/insights/logs/0HN29JEG2FM2Q:0000000F',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2024, 3, 21, 17, 11, 34, 897967, tzinfo=tzlocal())}


### Create corporate actions source

In this section we will create corporate action source and add this source to the transaction portfolio that we will create later on. 

Details around corporate actions in https://support.lusid.com/knowledgebase/article/KA-02017/en-us


In [ ]:
corporate_action_sql=f"""
@@corporateActionSourceScope = select 'FBN_Funds';
@@corporateActionSourceCode = select 'corporate-act';

/*

Step 1: Define a table containing sample information.

*/

@corporate_action_source_table =
select  
"corporate action" as DisplayName,
"corporate action source" as Description,
@@corporateActionSourceScope as CorporateActionSourceScope,
@@corporateActionSourceScope as InstrumentScopes,
@@corporateActionSourceCode as CorporateActionSourceCode,
"Insert" as WriteAction;

/*

Step 2: Write the corporate action source to LUSID.

*/

select * 
from Lusid.CorporateAction.Source.Writer
where ToWrite = @corporate_action_source_table;

"""
run_luminesce(corporate_action_sql)

,CorporateActionSourceScope,CorporateActionSourceCode,Description,DisplayName,InstrumentScopes,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,corporate-act,corporate action source,corporate action,FBN_Funds,Insert,2024-04-04 09:00:11.503,0,NaN,NaN


### Create portfolio

In this section we load a Transaction portfolio into LUSID.

In [ ]:
portfolio_load_sql = f"""
@@scope = select 'FBN_Funds';
@@portfolioCode = select 'FBN_Fund';
@@writeAction = select 'Upsert';
@@description = select 'FBN Fund';

@createPortfolio = select 'Transaction' as PortfolioType,
@@scope as PortfolioScope,
@@portfolioCode as PortfolioCode,
@@scope  as InstrumentScopes,
@@description as DisplayName,
@@portfolioCode  as Description,
'FBN_Funds' as CorporateActionSourceScope,
'corporate-act' as CorporateActionSourceId,
#2024-01-01# as Created,
'' as SubHoldingKeys,
'GBP' as BaseCurrency,
@@writeAction as WriteAction
;


select *
from Lusid.Portfolio.Writer
where ToWrite = @createPortfolio;
"""
run_luminesce(portfolio_load_sql)

,PortfolioScope,PortfolioCode,PortfolioType,BaseCurrency,DisplayName,Description,ParentPortfolioScope,ParentPortfolioCode,CorporateActionSourceScope,CorporateActionSourceId,...,Created,InstrumentScopes,AmortisationMethod,TransactionTypeScope,CashGainLossCalculationDate,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN_Fund,Transaction,GBP,FBN Fund,FBN_Fund,NaN,NaN,FBN_Funds,corporate-act,...,2024-01-01,FBN_Funds,NoAmortisation,default,Default,Insert,2024-04-04 09:00:22.565,0,NaN,NaN


### Create instrument

In this section we load some equity and bond instruments into LUSID.

In [ ]:
equity_instrument_load_sql = f"""
@@scope = select 'default';

@instrumentsData= 
values
('BP', 'FBNABOR001', 'GBP'),
('HSBC PLC', 'FBNABOR002', 'GBP'),
('Vodafone Group', 'FBNABOR003', 'GBP'),
('Microsoft Corp', 'FBNABOR004', 'USD'),
('Lloyds Banking Group', 'FBNABOR005', 'GBP');

@instrumentsForUpload = select
column1 as DisplayName,
column2 as ClientInternal,
column3 as DomCcy,
@@scope as Scope
from @instrumentsData;

-- Step 2: Upload the transformed data into LUSID

select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @instrumentsForUpload;
"""
run_luminesce(equity_instrument_load_sql)


,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail,WriteAction,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,...,AsAtCreated,UserIdCreated,RequestIdCreated,AsAtModified,UserIdModified,RequestIdModified,AsAtVersionNumber,Identifiers,DomCcy,LotSize
0,2024-04-02 15:28:17.156,0,NaN,NaN,Upsert,LUID_00003DBU,NaN,NaN,NaN,NaN,...,2024-04-02 15:28:17.156,00urv1yl4m7stbMC42p7,54b02c64-d17c-4562-b731-fb19960df0e3,2024-04-02 15:28:17.156,00urv1yl4m7stbMC42p7,54b02c64-d17c-4562-b731-fb19960df0e3,1,NaN,GBP,1
1,2024-04-02 15:28:17.156,0,NaN,NaN,Upsert,LUID_00003DBV,NaN,NaN,NaN,NaN,...,2024-04-02 15:28:17.156,00urv1yl4m7stbMC42p7,54b02c64-d17c-4562-b731-fb19960df0e3,2024-04-02 15:28:17.156,00urv1yl4m7stbMC42p7,54b02c64-d17c-4562-b731-fb19960df0e3,1,NaN,GBP,1
2,2024-04-02 15:28:17.156,0,NaN,NaN,Upsert,LUID_00003DBS,NaN,NaN,NaN,NaN,...,2024-04-02 15:28:17.156,00urv1yl4m7stbMC42p7,54b02c64-d17c-4562-b731-fb19960df0e3,2024-04-02 15:28:17.156,00urv1yl4m7stbMC42p7,54b02c64-d17c-4562-b731-fb19960df0e3,1,NaN,GBP,1
3,2024-04-02 15:28:17.156,0,NaN,NaN,Upsert,LUID_00003DBW,NaN,NaN,NaN,NaN,...,2024-04-02 15:28:17.156,00urv1yl4m7stbMC42p7,54b02c64-d17c-4562-b731-fb19960df0e3,2024-04-02 15:28:17.156,00urv1yl4m7stbMC42p7,54b02c64-d17c-4562-b731-fb19960df0e3,1,NaN,GBP,1
4,2024-04-02 15:28:17.156,0,NaN,NaN,Upsert,LUID_00003DBT,NaN,NaN,NaN,NaN,...,2024-04-02 15:28:17.156,00urv1yl4m7stbMC42p7,54b02c64-d17c-4562-b731-fb19960df0e3,2024-04-02 15:28:17.156,00urv1yl4m7stbMC42p7,54b02c64-d17c-4562-b731-fb19960df0e3,1,NaN,USD,1


In [ ]:
bond_instrument_load_sql = f"""
@@scope = select 'default';

-- Step 1: Define the bond instruments

@bondsData= 
values
('WELLTR 2.517 02/07/2118', 'FBNBND001',  '2023-01-01',  2.517, 'GBP', '1Y', 'ActAct', '2118-07-02'),
('CAMBRG 2.35 06/27/2078', 'FBNBND002',  '2023-01-01',  2.35, 'GBP', '1Y', 'ActAct', '2078-01-01'),
('HTHROW 4 3/8 03/01/27', 'FBNBND003',  '2023-01-01',  4, 'GBP', '1Y', 'ActAct', '2027-03-01'),
('RABOBK 5 3/8 08/03/60', 'FBNBND004',  '2023-01-01',  5, 'GBP', '1Y', 'ActAct', '2060-08-03'),
('COOPWH 6 1/4 07/08/26', 'FBNBND005',  '2023-01-01',  6, 'GBP', '1Y', 'ActAct', '2026-07-08')
;


@bondsDataForUpload = 
select
Column1 as DisplayName,
column2 as ClientInternal,
column3 as StartDate,
column4 as CouponRate,
column5 as DomCcy,
column5 as FlowConventionsCurrency,
column6 as FlowConventionsPaymentFrequency,
column7 as FlowConventionsDayCountConvention,
'MF' as FlowConventionsRollConvention,
column5 as FlowConventionsPaymentCalendars,
column5 as FlowConventionsResetCalendars,
0 as FlowConventionsSettleDays,
0 as FlowConventionsResetDays,
1 as Principal,
column8 as MaturityDate,
@@scope as Scope
from @bondsData;

-- Step 2: Upload the transformed data into LUSID

select *
from Lusid.Instrument.Bond.Writer
where ToWrite = @bondsDataForUpload;
"""
run_luminesce(bond_instrument_load_sql)

,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail,WriteAction,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,...,CalculationType,RoundingConventions1FaceValue,RoundingConventions1Precision,RoundingConventions1RoundingTarget,RoundingConventions1RoundingType,RoundingConventions2FaceValue,RoundingConventions2Precision,RoundingConventions2RoundingTarget,RoundingConventions2RoundingType,OriginalIssuePrice
0,2024-04-04 09:20:22.165,0,NaN,NaN,Upsert,LUID_00003DBX,NaN,NaN,NaN,NaN,...,Standard,0,0,NaN,NaN,0,0,NaN,NaN,NaN
1,2024-04-04 09:20:22.165,0,NaN,NaN,Upsert,LUID_00003DBY,NaN,NaN,NaN,NaN,...,Standard,0,0,NaN,NaN,0,0,NaN,NaN,NaN
2,2024-04-04 09:20:22.165,0,NaN,NaN,Upsert,LUID_00003DBZ,NaN,NaN,NaN,NaN,...,Standard,0,0,NaN,NaN,0,0,NaN,NaN,NaN
3,2024-04-04 09:20:22.165,0,NaN,NaN,Upsert,LUID_00003DC0,NaN,NaN,NaN,NaN,...,Standard,0,0,NaN,NaN,0,0,NaN,NaN,NaN
4,2024-04-04 09:20:22.165,0,NaN,NaN,Upsert,LUID_00003DC1,NaN,NaN,NaN,NaN,...,Standard,0,0,NaN,NaN,0,0,NaN,NaN,NaN


In [9]:
ibond_instrument_load_sql = f"""
@@scope = select 'default';

-- Step 1: Define the bond instruments

@ibondsData= 
values
('T2IL 2 IL TREASURY 35', 'FBNIBND001',  '2004-12-13',  2, 'GBP', '6M', 'ActAct', '2035-01-26','GBP','1M','UKRPI','6M','GB0031790826',100,'true'),
('TR400 5/8 IL TREASURY 40', 'FBNIBND002',  '2010-01-27',  7.037, 'GBP', '6M', 'ActAct', '2040-03-22','GBP','1M','UKRPI','6M','GB00B3LZBF68',100,'false')
;

@ibondsDataForUpload = 
select
Column1 as DisplayName,
column2 as ClientInternal,
column3 as StartDate,
column4 as CouponRate,
column5 as DomCcy,
column5 as FlowConventionsCurrency,
column6 as FlowConventionsPaymentFrequency,
column7 as FlowConventionsDayCountConvention,
'NoAdjustment' as FlowConventionsRollConvention,
column5 as FlowConventionsPaymentCalendars,
column5 as FlowConventionsResetCalendars,
0 as FlowConventionsSettleDays,
0 as FlowConventionsResetDays,
10000 as Principal,
column8 as MaturityDate,
column9 as InflationIndexConventionsCurrency,
column10 as InflationIndexConventionsInflationFrequency,
column11 as InflationIndexConventionsInflationIndexName,
column12 as InflationIndexConventionsObservationLag,
column13 as Isin,
column14 as principal,
column15 as principalProtection,
@@scope as Scope
from @ibondsData;

-- Step 2: Upload the transformed data into LUSID

select *
from Lusid.Instrument.InflationLinkedBond.Writer
where ToWrite = @ibondsDataForUpload;
"""
run_luminesce(ibond_instrument_load_sql)

,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail,WriteAction,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,...,PrincipalProtection,StubType,RoundingConventions1FaceValue,RoundingConventions1Precision,RoundingConventions1RoundingTarget,RoundingConventions1RoundingType,RoundingConventions2FaceValue,RoundingConventions2Precision,RoundingConventions2RoundingTarget,RoundingConventions2RoundingType
0,2024-04-16 10:48:58.139,0,NaN,NaN,Upsert,LUID_00003DCB,GB0031790826,NaN,NaN,NaN,...,True,ShortFront,0,0,NaN,NaN,0,0,NaN,NaN
1,2024-04-16 12:19:37.477,0,NaN,NaN,Upsert,LUID_00003DCC,GB00B3LZBF68,NaN,NaN,NaN,...,True,ShortFront,0,0,NaN,NaN,0,0,NaN,NaN


### Transactions

In this section we create some transactions in this portfolio for the instruments we just created.

In [ ]:
transaction_sql = f"""

@@scope = select 'FBN_Funds';
@@portfolioCode = select 'FBN_Fund';

-- Step 1: Define some transactions

@transactions = 
values

-- Equity Transactions
(@@scope, @@portfolioCode, 'txn_001', 'Buy', '2024-01-12', '2024-01-12', 10000, 161.02, 1610200, 'GBP', 'FBNABOR001', 1),
(@@scope, @@portfolioCode, 'txn_003', 'Buy', '2024-01-12', '2024-01-12', 10000, 16.582, 65820,'GBP', 'FBNABOR002', 1),
(@@scope, @@portfolioCode, 'txn_004', 'Buy', '2024-01-12', '2024-01-12', 10000, 77.35, 773500, 'GBP', 'FBNABOR003', 1),
(@@scope, @@portfolioCode, 'txn_011', 'Buy', '2024-01-12', '2024-01-12', 10000, 330, 3300000, 'USD', 'FBNABOR004', 1),
(@@scope, @@portfolioCode, 'txn_002', 'Buy', '2024-01-12', '2024-01-12', 1000, 46,46000, 'GBP', 'FBNABOR005', 1),

--Bond transactions
(@@scope, @@portfolioCode, 'txn_006', 'Buy', '2024-01-12', '2024-01-12', 10, 55, 55682.6, 'GBP', 'FBNBND001', 1),
(@@scope, @@portfolioCode, 'txn_005', 'Buy', '2024-01-12', '2024-01-12', 10, 66, 66000, 'GBP', 'FBNBND002', 1),
(@@scope, @@portfolioCode, 'txn_009', 'Buy', '2024-01-12', '2024-01-12', 10, 90, 90767.12, 'GBP', 'FBNBND003', 1),
(@@scope, @@portfolioCode, 'txn_007', 'Buy', '2024-01-12', '2024-01-12', 10, 90, 90986.64, 'GBP', 'FBNBND004', 1),
(@@scope, @@portfolioCode, 'txn_008', 'Buy', '2024-01-12', '2024-01-12', 10, 98, 98592.47, 'GBP', 'FBNBND005', 1)

;


-- Step 2: Load transactions into LUSID

@createTransactions = 
select
column1 as PortfolioScope,
column2 as PortfolioCode,
column3 as TxnId,
column4 as Type,
column5 as TransactionDate,
column6 as SettlementDate,
column7 as Units,
column8 as TradePrice,
column9 as TotalConsideration,
column10 as SettlementCurrency,
column11 as ClientInternal,
column12 as TradeToPortfolioRate,
'abor' as Source
from @transactions;

-- Upload the transformed data into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @createTransactions;
"""
run_luminesce(transaction_sql)

,PortfolioScope,PortfolioCode,DisplayName,TxnId,Type,TransactionDate,SettlementDate,Units,TradePrice,TradePriceType,...,ShareClassFigi,Wertpapier,RIC,QuotePermId,EdiKey,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN_Fund,FBN Fund,txn_001,Buy,2024-01-12,2024-01-12,10000,161.020,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-04-04 09:21:50.491,0,NaN,NaN
1,FBN_Funds,FBN_Fund,FBN Fund,txn_003,Buy,2024-01-12,2024-01-12,10000,16.582,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-04-04 09:21:50.491,0,NaN,NaN
2,FBN_Funds,FBN_Fund,FBN Fund,txn_004,Buy,2024-01-12,2024-01-12,10000,77.350,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-04-04 09:21:50.491,0,NaN,NaN
3,FBN_Funds,FBN_Fund,FBN Fund,txn_011,Buy,2024-01-12,2024-01-12,10000,330.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-04-04 09:21:50.491,0,NaN,NaN
4,FBN_Funds,FBN_Fund,FBN Fund,txn_002,Buy,2024-01-12,2024-01-12,1000,46.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-04-04 09:21:50.491,0,NaN,NaN
5,FBN_Funds,FBN_Fund,FBN Fund,txn_006,Buy,2024-01-12,2024-01-12,10,55.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-04-04 09:21:50.491,0,NaN,NaN
6,FBN_Funds,FBN_Fund,FBN Fund,txn_005,Buy,2024-01-12,2024-01-12,10,66.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-04-04 09:21:50.491,0,NaN,NaN
7,FBN_Funds,FBN_Fund,FBN Fund,txn_009,Buy,2024-01-12,2024-01-12,10,90.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-04-04 09:21:50.491,0,NaN,NaN
8,FBN_Funds,FBN_Fund,FBN Fund,txn_007,Buy,2024-01-12,2024-01-12,10,90.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-04-04 09:21:50.491,0,NaN,NaN
9,FBN_Funds,FBN_Fund,FBN Fund,txn_008,Buy,2024-01-12,2024-01-12,10,98.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-04-04 09:21:50.491,0,NaN,NaN


## Adding corporate actions

In this section we will add cash dividend and stock split to the BP stock which will occur 10 days after we acquire this stock. 

In [ ]:
corp_action_sql=f"""
@@corporateActionSourceScope = select 'FBN_Funds';
@@corporateActionSourceCode = select 'corporate-act';


-- Step 1: Define some corporate actions


@corpactions = 
values

-- BP corporate actions
('001', @@corporateActionSourceCode, @@corporateActionSourceScope, '2024-01-23', 'default', 'LUID_00003DBS', 'default', 'CCY_GBP', '2024-01-20', 'BP dividend', 1, 1, 0.1,0,'2024-01-20','2024-01-23'),
('002', @@corporateActionSourceCode, @@corporateActionSourceScope, '2024-01-23', 'default', 'LUID_00003DBS', 'default','LUID_00003DBS','2024-01-20', 'BP stock split', 1, 1, 2,1,'2024-01-20','2024-01-23')
;

@corporate_action_table =
select  
column1 as CorporateActionCode,
column2 as CorporateActionSourceCode,
column3 as CorporateActionSourceScope,
column4 as ExDate,
column5 as InputInstrumentScope,
column6 as InputLusidInstrumentId,
column7 as OutputInstrumentScope,
column8 as OutputLusidInstrumentId,
column9 as AnnouncementDate,
column10 as Description,
column11 as InputCostFactor,
column12 as InputUnitsFactor,
column13 as OutputUnitsFactor,
column14 as OutputCostFactor,
column15 as RecordDate,
column16 as PaymentDate,
"Upsert" as WriteAction
from @corpactions;



select * 
from Lusid.CorporateAction.Writer
where ToWrite = @corporate_action_table;

"""
run_luminesce(corp_action_sql)

,CorporateActionSourceScope,CorporateActionSourceCode,CorporateActionCode,Description,AnnouncementDate,ExDate,RecordDate,PaymentDate,InputInstrumentScope,InputLusidInstrumentId,...,InputCostFactor,OutputInstrumentScope,OutputLusidInstrumentId,OutputUnitsFactor,OutputCostFactor,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,corporate-act,1,BP dividend,2024-01-20,2024-01-23,2024-01-20,2024-01-23,default,LUID_00003DBS,...,1,default,CCY_GBP,0.1,0,Upsert,2024-04-04 09:00:11.503,0,NaN,NaN
1,FBN_Funds,corporate-act,2,BP stock split,2024-01-20,2024-01-23,2024-01-20,2024-01-23,default,LUID_00003DBS,...,1,default,LUID_00003DBS,2.0,1,Upsert,2024-04-04 09:00:11.503,0,NaN,NaN


### Quotes

In this section we insert some quotes for the instruments we just traded and also some FX rates for currency conversions in case the transactions are not in the currency of the portfolio.

In [ ]:
quote_sql = f"""

-- Load quotes from an Excel worksheet

@quotes_data = use Drive.Excel
--file=/abor-demo-data/prices/aborquotes.xlsx
--worksheet=prices
enduse;

-- Transform quote data

@quotes_for_upload = select
'ClientInternal' as InstrumentIdType,
client_internal as Instrumentid,
'FBN_Funds' as QuoteScope,
'Price' as QuoteType,
'Lusid' as Provider,
'Mid' as Field,
price_date as QuoteEffectiveAt,
close_price as Value,
unit as Unit
from @quotes_data;

-- Upload quotes into LUSID

select * from Lusid.Instrument.Quote.Writer
where ToWrite = @quotes_for_upload;
"""
run_luminesce(quote_sql)

,QuoteScope,Provider,PriceSource,InstrumentId,InstrumentIdType,QuoteType,Field,Lineage,ScaleFactor,QuoteEffectiveAt,Value,Unit,CutLabel,UploadedBy,QuoteAsAt,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,Lusid,NaN,FBNABOR005,ClientInternal,Price,Mid,NaN,NaN,2024-01-31,0.434200,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 12:06:32.828,Upsert,2024-03-25 12:06:32.828,0,NaN,NaN
1,FBN_Funds,Lusid,NaN,FBNBND004,ClientInternal,Price,Mid,NaN,NaN,2024-02-08,95.050003,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 12:06:32.828,Upsert,2024-03-25 12:06:32.828,0,NaN,NaN
2,FBN_Funds,Lusid,NaN,FBNABOR001,ClientInternal,Price,Mid,NaN,NaN,2024-01-15,4.652000,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 12:06:32.828,Upsert,2024-03-25 12:06:32.828,0,NaN,NaN
3,FBN_Funds,Lusid,NaN,FBNBND005,ClientInternal,Price,Mid,NaN,NaN,2024-02-15,79.639999,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 12:06:32.828,Upsert,2024-03-25 12:06:32.828,0,NaN,NaN
4,FBN_Funds,Lusid,NaN,FBNBND003,ClientInternal,Price,Mid,NaN,NaN,2024-01-23,93.849998,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 12:06:32.828,Upsert,2024-03-25 12:06:32.828,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,FBN_Funds,Lusid,NaN,FBNABOR003,ClientInternal,Price,Mid,NaN,NaN,2024-03-14,0.682100,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 12:06:32.828,Upsert,2024-03-25 12:06:32.828,0,NaN,NaN
558,FBN_Funds,Lusid,NaN,FBNBND003,ClientInternal,Price,Mid,NaN,NaN,2024-01-21,93.250000,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 12:06:32.828,Upsert,2024-03-25 12:06:32.828,0,NaN,NaN
559,FBN_Funds,Lusid,NaN,FBNBND005,ClientInternal,Price,Mid,NaN,NaN,2024-02-17,79.690002,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 12:06:32.828,Upsert,2024-03-25 12:06:32.828,0,NaN,NaN
560,FBN_Funds,Lusid,NaN,FBNABOR003,ClientInternal,Price,Mid,NaN,NaN,2024-01-25,0.674800,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 12:06:32.828,Upsert,2024-03-25 12:06:32.828,0,NaN,NaN


In [ ]:
fxrate_sql = f"""
@fx_price_data =
use Drive.Excel
--file=/abor-demo-data/prices/daily_quotes.xlsx
--worksheet=fx
enduse;

-- Run transformation on the data

@quotes_for_upload = select
'CurrencyPair' as InstrumentIdType,
ccy_pair as Instrumentid,
'FBN_Funds' as QuoteScope,
'Price' as QuoteType,
'Lusid' as Provider,
'Mid' as Field,
price_date as QuoteEffectiveAt,
close_price as Value,
'GBP' as Unit
from @fx_price_data;

-- Upload quotes into LUSID

select * from Lusid.Instrument.Quote.Writer
where ToWrite = @quotes_for_upload;
"""
run_luminesce(fxrate_sql)

,QuoteScope,Provider,PriceSource,InstrumentId,InstrumentIdType,QuoteType,Field,Lineage,ScaleFactor,QuoteEffectiveAt,Value,Unit,CutLabel,UploadedBy,QuoteAsAt,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Price,Mid,NaN,NaN,2024-01-16,1.271617,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 10:56:17.054,Upsert,2024-03-25 10:56:17.054,0,NaN,NaN
1,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Price,Mid,NaN,NaN,2024-01-24,1.269197,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 10:56:17.054,Upsert,2024-03-25 10:56:17.054,0,NaN,NaN
2,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Price,Mid,NaN,NaN,2024-02-08,1.262738,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 10:56:17.054,Upsert,2024-03-25 10:56:17.054,0,NaN,NaN
3,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Price,Mid,NaN,NaN,2024-02-26,1.267267,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 10:56:17.054,Upsert,2024-03-25 10:56:17.054,0,NaN,NaN
4,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Price,Mid,NaN,NaN,2024-01-15,1.273658,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 10:56:17.054,Upsert,2024-03-25 10:56:17.054,0,NaN,NaN
5,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Price,Mid,NaN,NaN,2024-01-17,1.263855,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 10:56:17.054,Upsert,2024-03-25 10:56:17.054,0,NaN,NaN
6,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Price,Mid,NaN,NaN,2024-02-09,1.262053,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 10:56:17.054,Upsert,2024-03-25 10:56:17.054,0,NaN,NaN
7,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Price,Mid,NaN,NaN,2024-02-28,1.268166,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 10:56:17.054,Upsert,2024-03-25 10:56:17.054,0,NaN,NaN
8,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Price,Mid,NaN,NaN,2024-02-12,1.263727,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 10:56:17.054,Upsert,2024-03-25 10:56:17.054,0,NaN,NaN
9,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Price,Mid,NaN,NaN,2024-02-21,1.262738,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-25 10:56:17.054,Upsert,2024-03-25 10:56:17.054,0,NaN,NaN


### Cash transactions

In this section we create some cash transactions

In [ ]:
cash_transaction_sql = f"""
@@scope = select 'FBN_Funds';
@@portfolioCode = select 'FBN_Fund_1';



/*  
    Step 1: Define some cash transactions
    
    In the step below, we create a transaction type called AborFundsIn which is configured to create
    Capital movements in the Journal Entry

*/

@transactions = 
values
(@@scope, @@portfolioCode, 'txn-0010', 'FundsIn', '2024-01-12', '2024-01-12', 20000000, 1, 20000000, 'GBP', 'CCY_GBP', 1, 1),

--Fx Spots
(@@scope, @@portfolioCode, 'TXN-25', 'FxSpotBuy', '2024-01-12', '2024-01-12', 3300000, 1.19, 2773109.24, 'USD', 'CCY_USD', 1, 1)
;




@createTransactions = 
select
column1 as PortfolioScope,
column2 as PortfolioCode,
column3 as TxnId,
column4 as Type,
column5 as TransactionDate,
column6 as SettlementDate,
column7 as Units,
column8 as TradePrice,
column9 as TotalConsideration,
column10 as SettlementCurrency,
column11 as LusidInstrumentId,
Column12 as TradeToPortfolioRate,
column13 as ExchangeRate,
'abor' as Source
from @transactions;

-- Step 2: Load transactions into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @createTransactions;
"""
run_luminesce(cash_transaction_sql)

,PortfolioScope,PortfolioCode,DisplayName,TxnId,Type,TransactionDate,SettlementDate,Units,TradePrice,TradePriceType,...,ShareClassFigi,Wertpapier,RIC,QuotePermId,EdiKey,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn-0010,FundsIn,2024-01-12,2024-01-12,20000000,1.00,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 16:15:15.033,0,NaN,NaN
1,FBN_Funds,FBN_Fund_1,FBN Fund 1,TXN-25,FxSpotBuy,2024-01-12,2024-01-12,3300000,1.19,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 16:15:15.033,0,NaN,NaN


### Chart of accounts
A chart of accounts (COA) is an index of all financial accounts in the general ledger of a company. We must create a COA before i can add the accounts. 

Since we have extra information to be shown as account properties, we will need to create the properties, inline them and add the accounts from the csv file. 


In [ ]:
chart_of_accounts_sql = f"""

-- Step 1: Define the Chart of Accounts

@@scope = select 'FBN_Funds';
@@code = select 'FBN_COA_1';
@@name = select 'FBN COA 1';
@@writeAction = select 'Upsert';

@chartOfAccounts =
select
@@scope as ChartOfAccountsScope,
@@code as ChartOfAccountsCode,
@@name as DisplayName,
@@name as Description,
@@writeAction as WriteAction;

-- Step 2: Upload Chart of Account into LUSID

select * from Lusid.ChartOfAccounts.Writer where ToWrite = @chartOfAccounts;
"""
run_luminesce(chart_of_accounts_sql)

,ChartOfAccountsScope,ChartOfAccountsCode,DisplayName,Description,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN_COA_1,FBN COA 1,FBN COA 1,Insert,2024-03-21 16:16:15.721,0,NaN,NaN


In [ ]:
add_acc_prop_sql = f"""

@@scope = select 'FBN_Funds';

-- Step 1: Define the property definitions

@newProperties =
values
('Account', @@scope, 'GL_Category','GL_Category', 'string'),
('Account', @@scope, 'GL_Type','GL_Type', 'string'),
('Account', @@scope, 'GL_Sub_Type','GL_Sub_Type', 'string'),
('Account', @@scope, 'Balance_Sheet','Balance Sheet', 'string'),
('Account', @@scope, 'PL_Statement','P/L Statement', 'string'),
('Account', @@scope, 'Statement_of_Net_Asset','Statement of Net Asset', 'string'),
('Account', @@scope, 'Income_VS_Capital_NAV','Income VS Capital NAV', 'string')
;

@propertyDefinitions =
select 
Column1 as [Domain], 
Column2 as [PropertyScope], 
Column3 as [PropertyCode], 
Column4 as [DisplayName], 
'Property' as [ConstraintStyle],
'system' as [DataTypeScope],
column5 as [DataTypeCode]
from @newProperties;

-- Step 2: Load property definitions

select *
from Lusid.Property.Definition.Writer
where ToWrite = @propertyDefinitions;


-- Step 3: Configure an entity provider pair to inline properties 
-- in order to interact with them the same way as standard entity data fields

@keysToCatalog = values
('Account/FBN_Funds/GL_Category', 'GL_Category', True, 'GL_Category'),
('Account/FBN_Funds/GL_Type', 'GL_Type', True, 'GL_Type'),
('Account/FBN_Funds/GL_Sub_Type', 'GL_Sub_Type', True, 'GL_Sub_Type'),
('Account/FBN_Funds/Balance_Sheet', 'Balance_Sheet', True, 'Balance Sheet'),
('Account/FBN_Funds/PL_Statement', 'PL_Statement', True, 'P/L Statement'),
('Account/FBN_Funds/Statement_of_Net_Asset', 'Statement_of_Net_Asset', True, 'Statement of Net Asset'),
('Account/FBN_Funds/Income_VS_Capital_NAV', 'Income_VS_Capital_NAV', True, 'Income VS Capital NAV')
;

@config = select column1 as [Key], column2 as Name, column3 as IsMain, column4 as Description from @keysToCatalog;

select * from Sys.Admin.Lusid.Provider.Configure
where Provider = 'Lusid.ChartOfAccounts.Account.Writer'
and Configuration = @config
and WriteAction = 'Modify';
"""
run_luminesce(add_acc_prop_sql)

,Domain,PropertyScope,PropertyCode,DisplayName,Description,Lifetime,ConstraintStyle,Required,DerivationFormula,DataTypeScope,DataTypeCode,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,Account,FBN_Funds,GL_Category,GL_Category,NaN,Perpetual,Property,False,NaN,system,string,Update,2024-04-03 15:30:50.720,0,NaN,NaN
1,Account,FBN_Funds,GL_Type,GL_Type,NaN,Perpetual,Property,False,NaN,system,string,Update,2024-04-03 15:30:50.720,0,NaN,NaN
2,Account,FBN_Funds,GL_Sub_Type,GL_Sub_Type,NaN,Perpetual,Property,False,NaN,system,string,Update,2024-04-03 15:30:50.720,0,NaN,NaN
3,Account,FBN_Funds,Balance_Sheet,Balance Sheet,NaN,Perpetual,Property,False,NaN,system,string,Update,2024-04-03 15:30:50.720,0,NaN,NaN
4,Account,FBN_Funds,PL_Statement,P/L Statement,NaN,Perpetual,Property,False,NaN,system,string,Update,2024-04-03 15:30:50.720,0,NaN,NaN
5,Account,FBN_Funds,Statement_of_Net_Asset,Statement of Net Asset,NaN,Perpetual,Property,False,NaN,system,string,Update,2024-04-03 15:30:50.720,0,NaN,NaN
6,Account,FBN_Funds,Income_VS_Capital_NAV,Income VS Capital NAV,NaN,Perpetual,Property,False,NaN,system,string,Update,2024-04-03 15:30:50.720,0,NaN,NaN


In [ ]:
add_accounts_sql = f"""

-- Load accounts from a csv worksheet

@coa_data = use Drive.Csv
--file=/abor-demo-data/COA/chart-of-accounts.csv
enduse;

-- Transform accounts data


@chartsOfAccountsUpload = select
'FBN_Funds' as ChartOfAccountsScope,
'FBN_COA_1' as ChartOfAccountsCode,
code as AccountCode,
description as Description,
type as Type,
'Manual' as Control,
GL_Category as GL_Category,
GL_Type as GL_Type,
GL_Sub_Type as GL_Sub_Type,
Balance_Sheet as Balance_Sheet,
PL_Statement as PL_Statement,
Statement_of_Net_Asset as Statement_of_Net_Asset,
Income_VS_Capital_NAV as Income_VS_Capital_NAV,
'Active' as Status
from @coa_data;

-- Step 2: Assign Accounts onto a ChartOfAccount

select * from Lusid.ChartOfAccounts.Account.Writer 
where ToWrite = @chartsOfAccountsUpload;
"""
run_luminesce(add_accounts_sql)

,ChartOfAccountsScope,ChartOfAccountsCode,AccountCode,Description,Type,Control,Status,GL_Category,GL_Type,GL_Sub_Type,Balance_Sheet,PL_Statement,Statement_of_Net_Asset,Income_VS_Capital_NAV,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN_COA_1,1_01_01_001,Equity - Cost,Asset,Manual,Active,1. Asset,1.01 Investment - Non Derivative,1.01.01 Valuation - Equity,YES,-,-,-,Upsert,2024-04-03 15:35:43.990,0,NaN,NaN
1,FBN_Funds,FBN_COA_1,1_01_01_002,Equity - Market Value Adjustment,Asset,Manual,Active,1. Asset,1.01 Investment - Non Derivative,1.01.01 Valuation - Equity,YES,-,-,-,Upsert,2024-04-03 15:35:44.683,0,NaN,NaN
2,FBN_Funds,FBN_COA_1,1_01_01_003,Equity - FX Adjusment,Asset,Manual,Active,1. Asset,1.01 Investment - Non Derivative,1.01.01 Valuation - Equity,YES,-,-,-,Upsert,2024-04-03 15:35:45.133,0,NaN,NaN
3,FBN_Funds,FBN_COA_1,1_01_02_001,Mutual Fund - Cost,Asset,Manual,Active,1. Asset,1.01 Investment - Non Derivative,1.01.02 Valuation - Mutual Fund,YES,-,-,-,Upsert,2024-04-03 15:35:45.589,0,NaN,NaN
4,FBN_Funds,FBN_COA_1,1_01_02_002,Mutual Fund - Market Value Adjustment,Asset,Manual,Active,1. Asset,1.01 Investment - Non Derivative,1.01.02 Valuation - Mutual Fund,YES,-,-,-,Upsert,2024-04-03 15:35:46.025,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,FBN_Funds,FBN_COA_1,5_04_02_009,Property Repairs / Maintenance,Expense,Manual,Active,5. Income & Expenses,5.04 Fees & Expenses,5.04.02 Other Expenses,-,YES,YES,Income NAV,Upsert,2024-04-03 15:38:08.234,0,NaN,NaN
305,FBN_Funds,FBN_COA_1,5_04_02_010,Surrender Premiums,Expense,Manual,Active,5. Income & Expenses,5.04 Fees & Expenses,5.04.02 Other Expenses,-,YES,YES,Income NAV,Upsert,2024-04-03 15:38:08.765,0,NaN,NaN
306,FBN_Funds,FBN_COA_1,5_04_02_011,TTF Holding Accounting Entries,Expense,Manual,Active,5. Income & Expenses,5.04 Fees & Expenses,5.04.02 Other Expenses,-,YES,YES,Income NAV,Upsert,2024-04-03 15:38:09.293,0,NaN,NaN
307,FBN_Funds,FBN_COA_1,5_04_02_012,Class Control,Expense,Manual,Active,5. Income & Expenses,5.04 Fees & Expenses,5.04.02 Other Expenses,-,YES,YES,Income NAV,Upsert,2024-04-03 15:38:09.823,0,NaN,NaN


## Create GL profile

Create GL profile and create mapping values.

In [ ]:
create_gl_profile_sql = f"""


@@scope = select 'FBN_Funds';
@@chartOfAccountsCode = select 'FBN_COA_1';
@@generalLedgerProfileCode = select 'FBN_GL';

@glProfile = select
@@scope as ChartOfAccountsScope,
@@chartOfAccountsCode as ChartOfAccountsCode,
@@chartOfAccountsCode as DisplayName,
@@generalLedgerProfileCode as GeneralLedgerProfileCode,
'Insert' as WriteAction
;

select * from Lusid.GeneralLedgerProfile.Writer
where ToWrite = @glProfile;
"""
run_luminesce(create_gl_profile_sql)

,ChartOfAccountsScope,ChartOfAccountsCode,GeneralLedgerProfileCode,Description,DisplayName,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN_COA_1,FBN_GL,NaN,FBN_COA_1,Insert,2024-04-04 08:30:53.094,0,NaN,NaN


### Define ABOR

Defining the ABOR configs. 

In [ ]:
create_abor_sql = f"""
@@scope = select 'FBN_Funds';
@@code = select 'standardAbor';
@@aborConfigCode = select 'standardAborConfiguration';
@@portfolioCode = select 'FBN_Fund_1';
@@writeAction = select 'Upsert';

-- Step 1: Define an ABOR

@aborForUpload = select
@@scope as AborScope,
@@code as AborCode,
@@code as DisplayName,
'SinglePortfolio' as PortfolioEntityType,
@@scope  as PortfolioScope,
@@portfolioCode as PortfolioCode,
@@portfolioCode as Description,
@@scope  as AborConfigurationScope,
@@aborConfigCode as AborConfigurationCode,
@@writeAction as WriteAction;

-- Step 2: Load ABOR into LUSID

select * from Lusid.Abor.Writer 
where ToWrite = @aborForUpload
"""
run_luminesce(create_abor_sql)

### Create a fund

Creating the fund as part of the ABOR specifications. 

In [ ]:
fund_sql = f"""

@fundData= 
values
('FBN1_Fund', 'FBN1 Fund', 'Finbourne T1 Fund', 'FBN_Funds', 'standardAbor','Standalone','2023-01-01', 31, 12,'FBN_Funds')
;
@createfund = 
select
column1 as FundCode,
column2 as DisplayName,
column3 as Description,
column4 as AborScope,
column5 as AborCode,
column6 as FundType,
column7 as InceptionDate,
column8 as YearEndDay,
column9 as YearEndMonth,
column10 as FundScope
from @fundData;

select * from Lusid.Fund.Writer
where ToWrite=@createfund;
"""
run_luminesce(fund_sql)

,FundScope,FundCode,FundType,DisplayName,ShareClassInstrumentScope,ShareClassInstrumentLusidInstrumentId,AborScope,AborCode,DecimalPlaces,YearEndDay,YearEndMonth,InceptionDate,Description,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN1_Fund,Standalone,FBN1 Fund,NaN,NaN,FBN_Funds,standardAbor,0,31,12,2023-01-01,Finbourne T1 Fund,Update,2024-03-25 11:21:15.706,0,NaN,NaN
